In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from koala.pointsets import uniform
from koala import graph_utils as gu
from koala import plotting as pl
from koala.voronization import generate_lattice
from koala.graph_color import color_lattice
from koala.flux_finder import fluxes_from_ujk, ujk_from_fluxes, loop_flux
from koala.lattice import Lattice
from scipy import linalg as la
from pfapack.ctypes import pfaffian
import pickle
from koala import example_graphs as eg
from dimer_models.koala_plantri import plantri_to_koala, plantri_generator, read_plantri
from dimer_models.lattice_generation import (
    bipartite_squarefull,
    reduce_bipartite,
)

from dimer_models.kasteleyn import (
    kasteleyn_matrix,
    find_kasteleyn_number,
    _fast_pfaffian,
    find_local_dimer_probability,
)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from copy import copy

cut_boundaries = False

ns = 30
lattice = gu.com_relaxation(generate_lattice(uniform(ns)))
lat_sq = gu.com_relaxation(bipartite_squarefull(ns))
lat_red = gu.com_relaxation(reduce_bipartite(lat_sq, 100))

lattices = [lattice, lat_sq, lat_red]

In [3]:
def _check_probabilities_sum(lattice, probs):
    sums = np.array([np.sum(probs[v]) for v in lattice.vertices.adjacent_edges])
    return sums
    assert np.allclose(sums, 1)


l = lat_red

norm = find_kasteleyn_number(l, False)

brute_force = np.mean(gu.dimerise(l, -1), axis=0)
probs = find_local_dimer_probability(l)

print(np.round(brute_force - probs, 4))
print(_check_probabilities_sum(l, probs))

[ 0.  0.  0. -0. -0.  0. -0. -0. -0. -0. -0. -0.  0. -0. -0. -0. -0. -0.
  0.  0.  0. -0. -0. -0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:
def _check_kast(lattice):

    all_dimers = gu.dimerise(lattice, -1)
    n_dimers = all_dimers.shape[0]

    n_kast = find_kasteleyn_number(lattice)
    n_kast_log = find_kasteleyn_number(lattice, True)

    # print(n_dimers, n_kast)
    if n_dimers > 0:

        kast_probs = find_local_dimer_probability(lattice)
        dimer_probabilities = np.mean(all_dimers, axis=0)

        if not np.allclose(dimer_probabilities, kast_probs):
            raise Exception("dimer probabilities is broken")
            sums = 1 - np.abs(_check_probabilities_sum(lattice, kast_probs))

            # fig, ax = plt.subplots()
            # pl.plot_edges(lattice, alpha=0.4)
            # p = lattice.vertices.positions
            # a = sums / np.max(sums)
            # ax.scatter(p[:, 0], p[:, 1], alpha=a)
            # plt.show()

    assert n_dimers == n_kast
    assert round(n_kast_log[0] * 10 ** n_kast_log[1]) == n_kast

In [10]:
# fig, ax = plt.subplots(1, 4, figsize=(10, 3))

with open("../tests/test_lattices", "rb") as f:
    while True:
        try:
            lattices = pickle.load(f)
            for l in lattices:
                _check_kast(l)

        except EOFError:
            break
        print()


# brute_force = np.mean(gu.dimerise(lat, -1), axis=0)
# kaseleyn_probs = find_local_dimer_probability(lat)

# print(kaseleyn_probs)
# # print(brute_force)
# print()
# # print(np.max(np.abs(kaseleyn_probs - brute_force)))

# pl.plot_edges(lat, ax=ax[j])
# ax[j].set_xticks([])
# ax[j].set_yticks([])
# ax[j].set_aspect("equal")

LinAlgError: singular matrix